In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchdiffeq import odeint
import warnings
warnings.filterwarnings("ignore")

In [4]:
def lorenz_system(state, t, sigma=10., beta=8./3, rho=28.):
    x, y, z = state
    dx = sigma * (y - x)
    dy = x * (rho - z) - y
    dz = x * y - beta * z
    return np.array([dx, dy, dz])

def simulate_lorenz(initial_state, timesteps, dt):
    from scipy.integrate import odeint as scipy_odeint
    t = np.linspace(0, timesteps*dt, timesteps)
    return scipy_odeint(lorenz_system, initial_state, t), t

In [6]:
# Generate data
initial = [1.0, 1.0, 1.0]
data, t = simulate_lorenz(initial, 10000, 0.01)
x = data[:, 0]  # 1D observation

# Create sliding windows
def create_sequences(x, window_size):
    X = []
    for i in range(len(x) - window_size):
        X.append(x[i:i+window_size])
    return torch.tensor(X, dtype=torch.float32)

window_size = 50
seq_data = create_sequences(x, window_size)


In [8]:
# Encoder
class Encoder(nn.Module):
    def __init__(self, input_size, latent_size):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, latent_size)
        )

    def forward(self, x):
        return self.fc(x)

# Latent Dynamics as Neural ODE
class ODEFunc(nn.Module):
    def __init__(self, latent_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_size, 64),
            nn.Tanh(),
            nn.Linear(64, latent_size)
        )

    def forward(self, t, z):
        return self.net(z)

# Decoder
class Decoder(nn.Module):
    def __init__(self, latent_size, output_size):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(latent_size, 64),
            nn.ReLU(),
            nn.Linear(64, output_size)
        )

    def forward(self, z):
        return self.fc(z)

In [10]:
# Full model
latent_dim = 3
encoder = Encoder(window_size, latent_dim)
odemodel = ODEFunc(latent_dim)
decoder = Decoder(latent_dim, 1)

params = list(encoder.parameters()) + list(odemodel.parameters()) + list(decoder.parameters())
optimizer = optim.Adam(params, lr=1e-3)

In [ ]:
# Training
epochs = 10
loss_fn = nn.MSELoss()

for epoch in range(epochs):
    epoch_loss = 0
    for i in range(len(seq_data) - 1):
        x_seq = seq_data[i]
        x_next = seq_data[i + 1][-1:]  # predict the next point

        z0 = encoder(x_seq)
        t_pred = torch.tensor([0., 1.], dtype=torch.float32)
        z_pred = odeint(odemodel, z0, t_pred)[-1]
        x_pred = decoder(z_pred)

        loss = loss_fn(x_pred, x_next)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {epoch_loss / len(seq_data):.6f}")

# Visualize latent space
z_latent = encoder(seq_data[:1000]).detach().numpy()
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.plot(z_latent[:, 0], z_latent[:, 1], z_latent[:, 2])
ax.set_title("Latent Space Trajectory")
plt.show()

Epoch 1, Loss: 9.788390
Epoch 2, Loss: 4.963071
Epoch 3, Loss: 5.272738


# Conclusion: Interpretation of Latent Space Trajectory

The plot above visualizes the trajectory of a data sequence in a learned 3D latent space. 
Each point on the curve represents a latent vector at a given timestep or sequence position, 
mapped from the original high-dimensional input (e.g., a frame, word, or state) into a compressed 
latent representation via an encoder or similar transformation.

Key observations:

1. The trajectory is continuous and smooth, indicating that the model has learned 
   structured latent representations rather than random noise.

2. The presence of curves and loops suggests that the model captures some temporal 
   or spatial patterns, typical in sequential data (e.g., time series, text, or video).

3. The scale across axes is not uniform (especially along the Y-axis), which may indicate:
   - Uneven variance in latent dimensions.
   - A need for normalisation or regularisation.
   - Or possibly a plotting issue (e.g., incorrect axis selection).

4. A sharp transition at the start may reflect a sudden change in input features or 
   initial model instability during encoding.

Overall, this trajectory provides insight into how the model internally represents 
the progression of input data. Further analysis or dimensionality reduction 
(e.g., PCA, t-SNE) could be used to refine interpretability and check clustering behaviour


-------------------------------------------------------------

### Lorenz Attractor Reconstruction

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Lorenz system differential equations
def lorenz(t, state, sigma=10.0, rho=28.0, beta=8.0/3.0):
    x, y, z = state
    dx_dt = sigma * (y - x)
    dy_dt = x * (rho - z) - y
    dz_dt = x * y - beta * z
    return [dx_dt, dy_dt, dz_dt]

# Initial conditions and time span
initial_state = [1.0, 1.0, 1.0]
t_span = (0, 40)
t_eval = np.linspace(t_span[0], t_span[1], 10000)

# Integrate the Lorenz equations
solution = solve_ivp(lorenz, t_span, initial_state, t_eval=t_eval)

# Plotting the Lorenz attractor
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.plot(solution.y[0], solution.y[1], solution.y[2], lw=0.5)
ax.set_title("Lorenz Attractor")
ax.set_xlabel("X Axis")
ax.set_ylabel("Y Axis")
ax.set_zlabel("Z Axis")
plt.show()

## 🧠 Conclusion

The **Lorenz attractor** is a classic example of a chaotic system. Even though it's defined by simple deterministic equations, it produces highly complex and non-repeating behavior.

This sensitivity to initial conditions — where tiny differences lead to vastly different outcomes — is a hallmark of **chaos theory**.

In this simulation, we used the standard Lorenz parameters:
- **σ (sigma)** = 10
- **ρ (rho)** = 28
- **β (beta)** = 8/3

These values create the iconic "butterfly" or double spiral pattern in 3D space.

> This visualization is more than just visually interesting — it's foundational in the study of **nonlinear dynamics**, **weather prediction**, and **chaotic systems** in physics and beyond.
